In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [ ]:
!pip install snowflake-connector-python


In [2]:
from snowflake.snowpark import Session

In [3]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [4]:
session = get_session()
# session.close()
session

connection_params =  {'user': 'PRAKHAR', 'password': 'Prak@apr24', 'account': 'ug94937.us-east4.gcp', 'warehouse': 'FOSFOR_INSIGHT_WH', 'database': 'test_database_prak', 'schema': 'test_schema_prak', 'role': 'PRAKHAR'}


In [5]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWSIGHT"

In [1]:
from snowflake.ml.registry import model_registry

In [ ]:
!pip freeze | grep -i cloud

In [6]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

create_model_registry() is in private preview since 0.2.0. Do not use it in production. 


True

In [10]:
registry1 = model_registry.Registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

AttributeError: module 'snowflake.ml.registry.model_registry' has no attribute 'Registry'

In [11]:
registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

<ipython-input-11-ac16ff576944>:1: DeprecationWarning: 
The `snowflake.ml.registry.model_registry.ModelRegistry` has been deprecated starting from version 1.2.0.
It will stay in the Private Preview phase. For future implementations, kindly utilize `snowflake.ml.registry.Registry`,
except when specifically required. The old model registry will be removed once all its primary functionalities are
fully integrated into the new registry.
        
  registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)


In [12]:
registry.get_history().show()

-------------------------------------------------------------------------------------------------------------------
|"EVENT_TIMESTAMP"  |"EVENT_ID"  |"MODEL_ID"  |"ROLE"  |"OPERATION"  |"ATTRIBUTE_NAME"  |"VALUE[ATTRIBUTE_NAME]"  |
-------------------------------------------------------------------------------------------------------------------
|                   |            |            |        |             |                  |                         |
-------------------------------------------------------------------------------------------------------------------



In [ ]:
import cloudpickle

model_obj = cloudpickle.load(open("/data/ml_model", "rb"))


In [ ]:
mv = registry.log_model(model_obj,
                   model_name="my_model",
                   version_name="v1",
                   conda_dependencies=["scikit-learn"],
                   comment="My awesome ML model",
#                    metrics={"score": 96},
                   sample_input_data=train_features)